## Connect to SAP BTP CloudFoundry and get a user key values

In [1]:
from cloudfoundry_client.client import CloudFoundryClient
import os

### Connect Python CF client -- option #1: using user and password

In [2]:
# Try to set CF user name from the OS environment variable CF_USERNAME
my_btp_user=os.environ.get('CF_USERNAME')
if not my_btp_user:
    my_btp_user='korystuvach@slava.ua'

In [ ]:
print(f"The user {my_btp_user} has been set.")

In [4]:
# Try to set CF user name from the OS environment variable CF_PASSWORD
my_btp_password=os.environ.get('CF_PASSWORD')

if not my_btp_password:
    import getpass
    # Prompt the user for password without displaying it
    my_btp_password = getpass.getpass(prompt='Enter your SAP BTP user password: ')


In [ ]:
print(f"The password {my_btp_password[0]+(len(my_btp_password)-1)*'*'} has been set.")

In [6]:
# Reuse the SAP BTP CloudFoundry API endpoint from your environment...
my_cf_target_endpoint = os.environ.get('CF_API_ENDPOINT')

# ...or set the API url by hand, if needed
my_cf_target_endpoint = 'https://api.cf.eu10-004.hana.ondemand.com'

In [7]:
proxy = dict(http=os.environ.get('HTTP_PROXY', ''), https=os.environ.get('HTTPS_PROXY', ''))
client = CloudFoundryClient(target_endpoint=my_cf_target_endpoint, proxy=proxy, verify=True)

client.init_with_user_credentials(login=my_btp_user, password=my_btp_password)

del my_btp_password

### Connect Python CF client -- option #2: using the default file, i.e. `~/.cf/config.json`

In [8]:
# use the default file, i.e. ~/.cf/config.json
client = CloudFoundryClient.build_from_cf_config(
    proxy = dict(http=os.environ.get('HTTP_PROXY', ''), https=os.environ.get('HTTPS_PROXY', ''))
)

### Get my SAP AI Core service instance

In [9]:
my_aicore_serivce_instance=client.v2.service_instances.get_first(name='vital-ai-core-extended')
display(my_aicore_serivce_instance['metadata'])

{'guid': '603c2b18-7729-466e-9b0c-762ba5f838c9',
 'url': '/v2/service_instances/603c2b18-7729-466e-9b0c-762ba5f838c9',
 'created_at': '2024-07-19T12:55:15Z',
 'updated_at': '2024-07-19T12:55:18Z'}

### Get my SAP AI Core service key

...where the service instance name in my SAP BTP subaccount is `vital-ai-core-extended`.

In [10]:
# You can use a specific service key, if you know the name, like 'vital-aicore-ext-sk1' in the example below
# my_cf_servicekey=client.v2.service_keys.get_first(name='vital-aicore-ext-sk1')

In [11]:
# Get the first found service key for the SAP AI Core service instance
my_cf_servicekey=client.v2.service_keys.get_first(service_instance_guid=my_aicore_serivce_instance['metadata']['guid'])

In [12]:
# Create a new service key `my_techbyte_sk`, if no service key found for the service instance
if not my_cf_servicekey:
    my_cf_servicekey=client.v2.service_keys.create(service_instance_guid=my_aicore_serivce_instance['metadata']['guid'], name='my_techbyte_sk')
    print(f"The new service key with the name `{my_cf_servicekey['entity']['name']}` has been created.")

The new service key with the name `my_techbyte_sk` has been created.


In [ ]:
display(my_cf_servicekey)

## Connect to my SAP AI Core instance

### Connect to my SAP AI Core instance -- option #1: using the values from the service key

In [14]:
# Load the AICore client
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create Connection
ai_core_client = AICoreV2Client(
    base_url      = my_cf_servicekey['entity']['credentials']['serviceurls']['AI_API_URL'] + '/v2', # The present SAP AI Core API version is 2
    auth_url      = my_cf_servicekey['entity']['credentials']['url'] + '/oauth/token', # Suffix to add
    client_id     = my_cf_servicekey['entity']['credentials']['clientid'],
    client_secret = my_cf_servicekey['entity']['credentials']['clientsecret'],
    resource_group='default'
)

### Connect to my SAP AI Core instance -- option #2: using the OS environment variables from the values from the service key

In [15]:
import os

os.environ.update({
    "AICORE_AUTH_URL":      f"{my_cf_servicekey['entity']['credentials']['url'] + '/oauth/token' }",
    "AICORE_CLIENT_ID":     f"{my_cf_servicekey['entity']['credentials']['clientid']}",
    "AICORE_CLIENT_SECRET": f"{my_cf_servicekey['entity']['credentials']['clientsecret']}",
    "AICORE_BASE_URL":      f"{my_cf_servicekey['entity']['credentials']['serviceurls']['AI_API_URL'] + '/v2'}",
    "AICORE_RESOURCE_GROUP":"default"
})

In [16]:
# Load the AICore client
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create Connection
ai_core_client = AICoreV2Client.from_env()

### Check GenAI model deployments in my SAP AI Core resource group

In [17]:
response = ai_core_client.deployment.query()

In [18]:
for deployment in response.resources:
    display(
        {key: deployment.__dict__[key] 
            for key in ('configuration_id', 'configuration_name', 'status', 'details') 
            if deployment.__dict__['scenario_id']=='foundation-models'}
    )


{'configuration_id': '14305345-7a79-45a0-a53a-1cd2f73613bb',
 'configuration_name': 'text-embedding-ada-002-config',
 'status': <Status.RUNNING: 'RUNNING'>,
 'details': {'resources': {'backend_details': {'model': {'name': 'text-embedding-ada-002',
     'version': 'latest'}}},
  'scaling': {'backend_details': {}}}}

{'configuration_id': 'df893fbf-5bae-4481-8a63-a572c83db2e6',
 'configuration_name': 'gpt-4-32k-config',
 'status': <Status.RUNNING: 'RUNNING'>,
 'details': {'resources': {'backend_details': {'model': {'name': 'gpt-4-32k',
     'version': 'latest'}}},
  'scaling': {'backend_details': {}}}}

## Use SAP Gen AI Hub SDK and instantiate one of the deployed Gen AI Models

In [19]:
from gen_ai_hub.proxy.langchain.init_models import init_llm

### Initiate GenAI model -- option #1: `init_llm()` implicitly uses OS environment variables to create its SAP Core AI client
 
It does not reuse the `ai_core_client` you created above.

In [20]:
llm = init_llm('gpt-4-32k', temperature=0., max_tokens=256)

### Initiate GenAI model -- option #2: explicitly reuse the `ai_core_client` in `init_llm()`

In [21]:
from gen_ai_hub.proxy.gen_ai_hub_proxy.client import GenAIHubProxyClient

genai_client=GenAIHubProxyClient(ai_core_client=ai_core_client)

In [22]:
llm = init_llm('gpt-4-32k', temperature=0., max_tokens=256, proxy_client=genai_client)

In [23]:
# Check the REST client is created
print(llm.proxy_client.ai_core_client.rest_client.base_url)
print(f"{llm.proxy_client.ai_core_client.rest_client.get_token()[:20]} ...")

https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2
Bearer eyJhbGciOiJSU ...


In [24]:
# Send a prompt to the LLM

llm_response=llm.invoke('Where is Wrocław located and what are top 5 places I should visit there?')

In [25]:
print(llm_response.content)
display(llm_response.response_metadata)

Wrocław is located in western Poland, on the banks of the River Oder. It is the largest city in the region of Lower Silesia.

Top 5 places to visit in Wrocław:

1. Market Square (Rynek): This is one of the largest market squares in Europe, filled with colorful buildings, restaurants, and shops. The Gothic-style Old Town Hall is a must-see.

2. Wrocław Cathedral: Also known as the Cathedral of St. John the Baptist, this is a stunning example of Gothic architecture and offers panoramic views of the city from its towers.

3. Wrocław's Dwarfs: Scattered throughout the city, these small figurines have become a symbol of Wrocław. There are over 350 dwarfs and finding them can be a fun activity for both kids and adults.

4. Panorama of Raclawice: This enormous painting depicts the Battle of Raclawice during the Kosciuszko Uprising. The painting is displayed in a special circular building and offers a 360-degree view.

5. Centennial Hall: This is a UNESCO World Heritage Site, known for its inn

{'token_usage': {'completion_tokens': 256,
  'prompt_tokens': 25,
  'total_tokens': 281},
 'model_name': 'gpt-4-32k',
 'system_fingerprint': None,
 'finish_reason': 'length',
 'logprobs': None}

## That's it for now!